<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#성장율이-1이-될때까지-돌림" data-toc-modified-id="성장율이-1이-될때까지-돌림-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>성장율이 1이 될때까지 돌림</a></span></li><li><span><a href="#프라타-과정" data-toc-modified-id="프라타-과정-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>프라타 과정</a></span></li></ul></div>

In [60]:
import pandas as pd

df = pd.read_excel('./500매트릭스.xlsx', index_col=0)
df10000 = pd.read_csv('./500기점.csv')
df10001 = pd.read_csv('./500종점.csv')

wntO = df10000['기점'].tolist()
wntD = df10001['종점'].tolist()

# df_copy = df.copy()

df

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,1,6,1,5,0,0,0,9,9,3,...,2,5,1,7,9,6,8,3,7,2166.0
1,9,2,7,1,3,9,1,8,5,7,...,3,5,1,7,2,7,3,1,3,2245.0
2,9,3,5,9,9,3,0,0,4,5,...,1,9,9,3,6,5,9,7,8,2236.0
3,8,5,1,3,8,7,4,7,5,4,...,2,6,7,7,7,3,5,6,2,2243.0
4,5,6,6,6,9,5,0,9,2,4,...,3,4,5,8,3,9,9,1,2,2309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,2,8,7,7,4,6,5,6,2,6,...,5,9,0,4,9,5,8,4,1,2320.0
497,3,7,3,7,8,5,9,9,6,9,...,1,6,9,0,5,8,3,7,2,2340.0
498,3,7,1,8,6,4,0,8,1,1,...,7,0,9,5,8,5,5,1,8,2203.0
499,6,2,5,9,4,4,3,1,3,1,...,5,8,5,9,7,5,2,9,7,2140.0


In [61]:
def level1(df):
    # 원하는 배열 입력
    # *wntO 와 wntD의 합은 같아야함

    # 기존 데이터 shape 알기
    df_copy = df.copy()
    value = df_copy.shape[0]

    # 기존 데이터 맨 가장자리에 원하는 값 넣어줌 (성장률 구할것)

    df[value] = pd.Series(wntO) # 시리즈 변환
    df.loc[value] = pd.Series(wntD) # .loc 는 row 

    # 성장률 구하기
    # 희망값/원래값
    df['Ei'] = df[value]/df[value-1] 
    df.loc['Fj'] = df.loc[value]/df.loc[value-1]
    
    df = df.fillna(0) # 결측치 값 0으로 변경 (nan 값 있으면 sum 같은 함수 적용 안됨)
    apple = []

    # 유출통행량 보정값
    
    for i in range(len(df)-3):
        value2 = df.iloc[i,value-1]/sum(df.loc[i] * df.loc['Fj'])
        apple.append(value2)

    df['Li'] = pd.Series(apple)
    df = df.fillna(0)

    banana = []

    # 유입통행량 보정값

    for i in range(len(df)-3):
        value3 = df.loc[value-1,i]/sum(df[i]*df['Ei'])
        banana.append(value3)

    df.loc['Lj'] = pd.Series(banana)
    df = df.fillna(0)
    
    return df, df_copy

In [62]:
def level2(df, df_copy):   
    result = df_copy.applymap(lambda x: None)
    value = df_copy.shape[0]
    
    i = ''
    j = ''

    for i in range(value-1):
        for j in range(value-1):
            #print(i, ',', j)
            result.loc[i,j] = df.loc[i,j]*df.loc[i,'Ei']*df.loc['Fj'][j]*(df.loc[i,'Li'] + df.loc['Lj'][j])/2

    result = result.fillna(0)
    #result = result.round(0) # 반올림
    result

    for num in range(value-1):
        result.loc[num,value-1] = sum(result.loc[num]) # 가로 합

    for num in range(value-1):
        result.loc[value-1,num] = sum(result[num]) # 세로 합

    return result

In [63]:
import time
start = time.time()

df_result, df_copy = level1(df)
result = level2(df_result, df_copy)
result

print('result')

cycle1,df_copy = level1(result)
cycle1 = level2(cycle1,df_copy)
cycle1

print('cycle1')

wntcyc = 10 # << 수정하셈

for s in range(2,wntcyc):
    print(s)
    globals()['cycle{}'.format(s)] = level1(globals()['cycle{}'.format(s-1)])[0]
    globals()['cycle{}'.format(s)] = level2(globals()['cycle{}'.format(s)], df_copy)
    globals()['cycle{}'.format(s)]
    
end = time.time()
print(f"{end - start:.5f} sec")

result
cycle1
2
3
4
5
6
7
8
9
436.29939 sec


500*500 하니까 7분이 걸려버리는군;;

### 성장율이 1이 될때까지 돌림

In [64]:
cycle9

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,0.690557,24.789962,2.019492,14.389372,0.000000,0.000000,0.000000,22.075179,13.576693,5.932097,...,1.491355,19.680445,3.467690,22.160398,47.265340,7.832040,30.346167,5.430401,31.633861,6403.0
1,3.564246,4.739265,8.107145,1.650418,0.750643,6.795107,1.004090,11.253225,4.325889,7.938128,...,1.282985,11.287077,1.988820,12.708888,6.023623,5.240137,6.526613,1.038137,7.775019,3844.0
2,3.384643,6.750568,5.499010,14.105303,2.138442,2.150863,0.000000,0.000000,3.286276,5.384353,...,0.406107,19.292760,16.997163,5.172195,17.160256,3.554355,18.592940,6.900710,19.688618,3627.0
3,3.363192,12.580412,1.229458,5.255808,2.125063,5.611793,4.262925,10.452501,4.593149,4.815514,...,0.908134,14.380451,14.782028,13.491469,22.380457,2.383889,11.549628,6.613133,5.502348,4218.0
4,2.295725,16.484252,8.056503,11.480702,2.610856,4.376848,0.000000,14.677564,2.006187,5.259055,...,1.487496,10.468985,11.529281,16.839371,10.475492,7.811058,22.700985,1.203613,6.009453,4602.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.265066,6.344686,2.713119,3.866210,0.334952,1.516165,1.680030,2.824463,0.579124,2.277086,...,0.715649,6.799549,0.000000,2.430376,9.071329,1.252583,5.824934,1.389753,0.867319,1346.0
497,1.075307,15.011366,3.144663,10.456424,1.811649,3.416351,8.179497,11.458192,4.697875,9.237144,...,0.387036,12.257960,16.198722,0.000000,13.629537,5.420352,5.906556,6.576758,4.691327,3576.0
498,1.421063,19.841026,1.385281,15.792419,1.795708,3.612434,0.000000,13.459953,1.034878,1.356411,...,3.580795,0.000000,21.410294,10.857976,28.819450,4.476912,13.011268,1.241746,24.799172,4538.0
499,4.945346,9.862537,12.051947,30.914453,2.082951,6.284766,6.269577,2.927573,5.401418,2.360098,...,4.449978,37.583109,20.694010,34.006837,43.877661,7.790070,9.054809,19.444283,37.757039,7520.0


### 프라타 과정

In [65]:
cycle1

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,Ei
0,0.690682,24.788295,2.019868,14.386410,0.000000,0.000000,0.000000,22.076903,13.580863,5.931982,...,3.466665,22.158174,47.267476,7.831821,30.347978,5.431986,31.631005,6402.725770,6403.0,1.000043
1,3.565257,4.739432,8.109487,1.650248,0.750942,6.796804,1.004314,11.255257,4.327661,7.938788,...,1.988437,12.708915,6.024513,5.240528,6.527671,1.038547,7.775114,3844.242144,3844.0,0.999937
2,3.385392,6.750386,5.500257,14.102968,2.139162,2.151266,0.000000,0.000000,3.287418,5.384466,...,16.992826,5.171884,17.161723,3.554399,18.594799,6.903002,19.687633,3627.019631,3627.0,0.999995
3,3.364939,12.583823,1.230103,5.256505,2.126413,5.614518,4.264884,10.456855,4.596114,4.817051,...,14.782662,13.494679,22.389042,2.384629,11.554226,6.617302,5.503713,4219.172806,4218.0,0.999722
4,2.296471,16.485516,8.059165,11.479991,2.612006,4.378123,0.000000,14.680822,2.007092,5.259710,...,11.527533,16.840105,10.477474,7.811963,22.705608,1.204138,6.009776,4602.471177,4602.0,0.999898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.421522,19.842515,1.385736,15.791414,1.796496,3.613480,0.000000,13.462918,1.035343,1.356578,...,21.407013,10.858431,28.824853,4.477424,13.013895,1.242285,24.800460,4538.440412,4538.0,0.999903
499,4.945620,9.860635,12.052679,30.904205,2.083307,6.284902,6.269548,2.927434,5.402398,2.359755,...,20.685296,33.999151,43.874131,7.788873,9.054211,19.447516,37.748887,7518.824646,7520.0,1.000156
500,1202.253479,6627.775533,3248.722090,4890.167085,746.251328,2163.414877,2995.466211,4113.484224,2490.838439,3347.096910,...,5441.699209,5152.697082,9477.662268,2203.024164,5996.588055,3136.034502,7576.564389,0.000000,NaN,NaN
501,1202.000000,6628.000000,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000000,3347.000000,...,5443.000000,5153.000000,9477.000000,2203.000000,5996.000000,3135.000000,7577.000000,NaN,NaN,NaN


In [66]:
cycle2

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,Ei
0,0.690562,24.790078,2.019499,14.389420,0.000000,0.000000,0.000000,22.075214,13.576789,5.932078,...,3.467665,22.160410,47.265778,7.832054,30.346234,5.430415,31.634016,6403.010111,6403.0,0.999998
1,3.564265,4.739281,8.107163,1.650422,0.750643,6.795093,1.004093,11.253228,4.325914,7.938094,...,1.988804,12.708878,6.023671,5.240140,6.526619,1.038139,7.775047,3843.999672,3844.0,1.000000
2,3.384652,6.750573,5.499008,14.105294,2.138436,2.150853,0.000000,0.000000,3.286286,5.384315,...,16.996973,5.172177,17.160347,3.554347,18.592907,6.900700,19.688636,3626.991384,3627.0,1.000002
3,3.363246,12.580588,1.229474,5.255875,2.125086,5.611841,4.262983,10.452615,4.593224,4.815544,...,14.782060,13.491602,22.380873,2.383915,11.549761,6.613212,5.502426,4218.043521,4218.0,0.999990
4,2.295739,16.484321,8.056528,11.480735,2.610859,4.376843,0.000000,14.677580,2.006200,5.259036,...,11.529192,16.839372,10.475585,7.811068,22.701025,1.203616,6.009480,4602.004384,4602.0,0.999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.421074,19.841146,1.385288,15.792493,1.795713,3.612436,0.000000,13.459992,1.034886,1.356409,...,21.410169,10.857996,28.819757,4.476926,13.011314,1.241751,24.799327,4538.012575,4538.0,0.999997
499,4.945348,9.862522,12.051914,30.914363,2.082940,6.284722,6.269564,2.927559,5.401422,2.360076,...,20.693731,34.006642,43.877794,7.790035,9.054772,19.444212,37.756988,7519.966263,7520.0,1.000004
500,1202.005807,6628.020612,3248.006106,4891.009887,745.999727,2162.994677,2995.007499,4112.999975,2490.012438,3346.985633,...,5442.954300,5152.995005,9477.070565,2203.000792,5996.003676,3135.002469,7577.025853,0.000000,NaN,NaN
501,1202.000000,6628.000000,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000000,3347.000000,...,5443.000000,5153.000000,9477.000000,2203.000000,5996.000000,3135.000000,7577.000000,NaN,NaN,NaN


In [67]:
cycle3

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,Ei
0,0.690557,24.789962,2.019492,14.389370,0.000000,0.000000,0.000000,22.075179,13.576697,5.932096,...,3.467689,22.160397,47.265347,7.832039,30.346167,5.430402,31.633860,6402.999711,6403.0,1.0
1,3.564248,4.739266,8.107147,1.650418,0.750643,6.795108,1.004090,11.253227,4.325891,7.938128,...,1.988820,12.708890,6.023625,5.240138,6.526614,1.038138,7.775020,3844.000551,3844.0,1.0
2,3.384643,6.750568,5.499010,14.105301,2.138442,2.150863,0.000000,0.000000,3.286277,5.384352,...,16.997159,5.172195,17.160258,3.554355,18.592940,6.900710,19.688617,3626.999789,3627.0,1.0
3,3.363194,12.580416,1.229459,5.255809,2.125064,5.611795,4.262927,10.452504,4.593151,4.815515,...,14.782030,13.491472,22.380467,2.383889,11.549632,6.613136,5.502349,4218.001100,4218.0,1.0
4,2.295725,16.484253,8.056505,11.480702,2.610857,4.376849,0.000000,14.677565,2.006187,5.259054,...,11.529279,16.839372,10.475495,7.811058,22.700987,1.203613,6.009454,4602.000180,4602.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.421063,19.841028,1.385281,15.792418,1.795708,3.612434,0.000000,13.459954,1.034878,1.356411,...,21.410291,10.857976,28.819457,4.476912,13.011269,1.241746,24.799173,4538.000197,4538.0,1.0
499,4.945346,9.862536,12.051946,30.914446,2.082951,6.284766,6.269576,2.927573,5.401418,2.360097,...,20.694003,34.006832,43.877663,7.790068,9.054808,19.444284,37.757034,7519.998976,7520.0,1.0
500,1202.000187,6628.000245,3248.000304,4890.999547,746.000058,2163.000250,2995.000270,4113.000177,2490.000689,3346.999594,...,5442.999176,5152.999973,9477.001567,2202.999929,5996.000302,3135.000504,7576.999967,0.000000,NaN,NaN
501,1202.000000,6628.000000,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000000,3347.000000,...,5443.000000,5153.000000,9477.000000,2203.000000,5996.000000,3135.000000,7577.000000,NaN,NaN,NaN


In [68]:
cycle4

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,Ei
0,0.690557,24.789962,2.019492,14.389372,0.000000,0.000000,0.000000,22.075179,13.576693,5.932097,...,3.467690,22.160399,47.265341,7.832040,30.346167,5.430401,31.633861,6403.000018,6403.0,1.0
1,3.564246,4.739265,8.107145,1.650418,0.750643,6.795107,1.004090,11.253225,4.325889,7.938128,...,1.988820,12.708888,6.023623,5.240137,6.526613,1.038137,7.775019,3844.000006,3844.0,1.0
2,3.384643,6.750568,5.499010,14.105303,2.138442,2.150863,0.000000,0.000000,3.286276,5.384353,...,16.997163,5.172195,17.160256,3.554355,18.592940,6.900710,19.688618,3626.999994,3627.0,1.0
3,3.363192,12.580412,1.229458,5.255809,2.125063,5.611793,4.262925,10.452501,4.593149,4.815514,...,14.782028,13.491469,22.380457,2.383889,11.549628,6.613133,5.502348,4218.000039,4218.0,1.0
4,2.295725,16.484252,8.056503,11.480702,2.610856,4.376848,0.000000,14.677564,2.006187,5.259055,...,11.529281,16.839371,10.475493,7.811058,22.700985,1.203613,6.009453,4602.000001,4602.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.421063,19.841027,1.385281,15.792419,1.795708,3.612434,0.000000,13.459953,1.034878,1.356411,...,21.410294,10.857976,28.819451,4.476912,13.011268,1.241746,24.799172,4538.000010,4538.0,1.0
499,4.945346,9.862537,12.051947,30.914453,2.082951,6.284766,6.269577,2.927573,5.401418,2.360098,...,20.694009,34.006837,43.877662,7.790070,9.054809,19.444283,37.757039,7519.999982,7520.0,1.0
500,1202.000006,6628.000039,3248.000000,4891.000010,745.999999,2162.999999,2995.000005,4112.999999,2490.000017,3346.999978,...,5442.999970,5152.999990,9477.000088,2202.999999,5996.000017,3135.000006,7577.000021,0.000000,NaN,NaN
501,1202.000000,6628.000000,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000000,3347.000000,...,5443.000000,5153.000000,9477.000000,2203.000000,5996.000000,3135.000000,7577.000000,NaN,NaN,NaN


In [69]:
cycle5

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,Ei
0,0.690557,24.789962,2.019492,14.389372,0.000000,0.000000,0.000000,22.075179,13.576693,5.932097,...,3.467690,22.160398,47.265340,7.832040,30.346167,5.430401,31.633861,6403.000000,6403.0,1.0
1,3.564246,4.739265,8.107145,1.650418,0.750643,6.795107,1.004090,11.253225,4.325889,7.938128,...,1.988820,12.708888,6.023623,5.240137,6.526613,1.038137,7.775019,3844.000001,3844.0,1.0
2,3.384643,6.750568,5.499010,14.105303,2.138442,2.150863,0.000000,0.000000,3.286276,5.384353,...,16.997163,5.172195,17.160256,3.554355,18.592940,6.900710,19.688618,3627.000000,3627.0,1.0
3,3.363192,12.580412,1.229458,5.255808,2.125063,5.611793,4.262925,10.452501,4.593149,4.815514,...,14.782028,13.491469,22.380457,2.383889,11.549628,6.613133,5.502348,4218.000001,4218.0,1.0
4,2.295725,16.484252,8.056503,11.480702,2.610856,4.376848,0.000000,14.677564,2.006187,5.259055,...,11.529281,16.839371,10.475492,7.811058,22.700985,1.203613,6.009453,4602.000000,4602.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.421063,19.841026,1.385281,15.792419,1.795708,3.612434,0.000000,13.459953,1.034878,1.356411,...,21.410294,10.857976,28.819450,4.476912,13.011268,1.241746,24.799172,4538.000000,4538.0,1.0
499,4.945346,9.862537,12.051947,30.914453,2.082951,6.284766,6.269577,2.927573,5.401418,2.360098,...,20.694010,34.006837,43.877661,7.790070,9.054809,19.444283,37.757039,7519.999999,7520.0,1.0
500,1202.000000,6628.000001,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000001,3346.999999,...,5442.999999,5153.000000,9477.000002,2203.000000,5996.000000,3135.000000,7577.000000,0.000000,NaN,NaN
501,1202.000000,6628.000000,3248.000000,4891.000000,746.000000,2163.000000,2995.000000,4113.000000,2490.000000,3347.000000,...,5443.000000,5153.000000,9477.000000,2203.000000,5996.000000,3135.000000,7577.000000,NaN,NaN,NaN


In [ ]:
## ... 기타 생략